In [1]:
import mlrun

# Initialize mlrun environment and load the project
from os import path
project_dir = "./project"
project_dir = path.abspath('conf')
project_name = "widsdb2"

wids_prj, artifact_path = mlrun.set_environment('http://mlrun-api:8080', 
                                                project=project_name, user_project=False)

widsdb2_proj = mlrun.projects.load_project(project_dir,  clone=True)



In [2]:
print(path.abspath('conf'))

/User/icuhack/conf


In [3]:
%nuclio cmd -c pip install lightgbm

In [4]:
%nuclio config spec.build.baseImage = "mlrun/mlrun"
%nuclio config kind = "serving"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'
%nuclio: setting kind to 'serving'


In [5]:
# nuclio: start-code

In [6]:
import numpy as np
from cloudpickle import load
import mlrun
class LGBMModel(mlrun.serving.V2ModelServer):
    
    def load(self):
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body):
        try:
            feats = np.asarray(body['inputs'])
            result = self.model.predict(feats)
            return result.tolist()
        except Exception as exc:
            raise Exception(f"Failed to predict {exc}")

In [7]:
# nuclio: end-code

In [8]:
import mlrun
from mlrun import code_to_function



serving_fn = mlrun.code_to_function('lightgbm-serving',
                            description="LightGBM Serving",
                            categories=['serving', 'ml'],
                            labels={'author': 'Aruna', 'framework': 'lightgbm'})
serving_fn.spec.default_class = 'LGBMModel'



In [9]:
tmodel = f'store://artifacts/widsdb2/train-wids-train_wids_model:latest'
        
                
modeldata = mlrun.get_dataitem(tmodel)
modeldata



'v3io:///projects/widsdb2/artifacts/'

In [10]:
serving_fn.add_model('lgbm_model',model_path=tmodel)

In [11]:
widsdb2_proj.set_function(serving_fn)

In [12]:
widsdb2_proj.save()

In [13]:
#widsdb2_proj.functions

In [14]:
#dfa = mlrun.get_dataitem('store://artifacts/widsdb2/prep-trdata_prep_train_enc').as_df()

dfa = mlrun.get_dataitem('store://widsdb2/tstprep-tstdata_prep_test_enc:latest').as_df()

dfa.head()



,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,...,d1_resprate_div_sysbp_min,d1_lactate_min_div_diasbp_min,d1_heartrate_min_div_d1_sysbp_min,d1_hco3_div,d1_resprate_times_resprate,left_average_spo2,total_chronic,total_cancer_immuno,has_complicator,apache_3j
0,72.0,0.0,0,2,0,152.4,3,0,82,0,...,0.197368,0.0,0.500000,1.130435,525.0,98.666667,0,0,0,0
1,86.0,0.0,0,2,0,175.3,2,0,82,0,...,0.215385,0.0,0.861538,1.000000,714.0,98.333333,0,0,0,0
2,72.0,0.0,0,2,0,162.6,3,1,82,0,...,0.077670,0.0,0.660194,1.032258,392.0,94.333333,0,0,0,0
3,66.0,0.0,0,2,1,177.8,3,1,82,0,...,0.200000,0.0,0.905263,1.000000,1064.0,92.666667,0,0,0,7
4,89.0,0.0,0,2,1,170.2,1,0,82,0,...,0.078947,0.0,0.298246,1.100000,441.0,97.666667,0,0,0,0


In [15]:
import json
my_data = dfa.values.tolist()
#my_data

In [16]:
tmodel

'store://artifacts/widsdb2/train-wids-train_wids_model:latest'

In [17]:
serving_fn.apply(mlrun.platforms.auto_mount())
address = serving_fn.deploy()

> 2021-07-01 12:45:46,338 [info] Starting remote function deploy
2021-07-01 12:45:47  (info) Deploying function
2021-07-01 12:45:47  (info) Building
2021-07-01 12:45:47  (info) Staging files and preparing base images
2021-07-01 12:45:47  (info) Building processor image
2021-07-01 12:45:50  (info) Build complete
2021-07-01 12:45:58  (info) Function deploy complete
> 2021-07-01 12:45:59,844 [info] function deployed, address=default-tenant.app.mlops1.iguazio-c0.com:31615


In [18]:
## Run the inference with the serving function
server = serving_fn.to_mock_server()

server.test("/v2/models/lgbm_model/infer", body={"inputs": my_data})

> 2021-07-01 12:46:41,986 [info] model lgbm_model was loaded
> 2021-07-01 12:46:41,987 [info] Loaded ['lgbm_model']


{'id': 'd178da938059439ea094bec6cc9655a2',
 'model_name': 'lgbm_model',
 'outputs': [0.19205241046239535,
  0.1867916230249813,
  0.18780778357088107,
  0.18539538889397503,
  0.20248756927625264,
  0.18255879699007505,
  0.20068102260298426,
  0.18542650528945065,
  0.2318329126831736,
  0.24183774315724296,
  0.17616607633462653,
  0.192232476583831,
  0.18283841552061653,
  0.20930142691095843,
  0.22784624167476014,
  0.17607924104557482,
  0.17681146160446468,
  0.22064169147471793,
  0.21825133433410976,
  0.1917688722633487,
  0.19049805265401018,
  0.19397175244891057,
  0.20741383612375072,
  0.22888756141371766,
  0.18028349446138625,
  0.23705080759851666,
  0.21041870901903093,
  0.20598388663903489,
  0.24137400818017227,
  0.18603952622316366,
  0.19192783548442666,
  0.17673492159380258,
  0.17679310127698133,
  0.2200348706792738,
  0.20030906496138068,
  0.20087499248979693,
  0.18828545377329592,
  0.18834106548555343,
  0.18875147285003824,
  0.20136428080308055,
  0